In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('Training_Data/overall_training_data.csv',index_col=0)
df

,Player,Pos,Age,G,MP,PER,TS%,3PAr,FTr,ORB%,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,All Star
0,Quincy Acy,PF,24,68,1287,11.9,0.533,0.181,0.293,6.9,...,15.5,1.0,0.7,1.7,0.063,-2.1,-1.0,-3.1,-0.3,0
1,Jordan Adams,SG,20,30,248,12.8,0.489,0.291,0.267,4.2,...,20.4,0.0,0.4,0.4,0.073,-1.2,2.6,1.4,0.2,0
2,Steven Adams,C,21,70,1771,14.1,0.549,0.005,0.514,12.2,...,14.3,1.9,2.2,4.1,0.111,-1.5,0.2,-1.3,0.3,0
3,Jeff Adrien,PF,28,17,215,14.2,0.494,0.000,0.864,11.9,...,14.3,0.2,0.2,0.4,0.087,-1.9,-0.6,-2.4,0.0,0
4,Arron Afflalo,SG,29,78,2502,10.7,0.533,0.377,0.224,1.1,...,19.0,1.6,1.0,2.6,0.050,-1.4,-1.1,-2.5,-0.3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
646,Trae Young,PG,21,60,2120,23.9,0.595,0.455,0.448,1.6,...,34.9,5.3,0.6,5.9,0.133,6.2,-2.3,3.9,3.1,1
647,Cody Zeller,C,27,58,1341,18.8,0.576,0.157,0.374,12.6,...,20.8,2.3,1.3,3.6,0.129,0.2,-0.8,-0.6,0.5,0
648,Tyler Zeller,C,30,2,4,22.4,0.250,0.000,0.000,80.9,...,43.2,0.0,0.0,0.0,-0.075,-0.3,-22.1,-22.4,0.0,0
649,Ante Žižić,C,23,22,221,16.4,0.597,0.000,0.264,9.0,...,17.5,0.3,0.2,0.5,0.106,-1.7,-1.5,-3.2,-0.1,0
